# 高级控制教程：结合预测与MPC实现智能调度

在本教程中，我们将演示如何将预测模型与模型预测控制（MPC）相结合，以实现比传统控制器更优越的系统性能。

我们将对比两种控制策略：
1.  **PID控制器**: 一种经典的的被动式反馈控制器，它只根据当前的误差（例如，当前水位与目标水位的差距）来做出反应。
2.  **MPC控制器**: 一种先进的主动式控制策略，它利用对未来的预测来提前规划最优的控制动作。

**目标**: 证明在有显著外部扰动（如变化的入库流量）的情况下，MPC能够更好地维持水库水位的稳定。

### 准备工作

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 导入平台组件
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.mpc_controller import MPCController
from swp.local_agents.control.local_control_agent import LocalControlAgent

# 设置 Matplotlib 样式
plt.style.use('seaborn-v0_8-whitegrid')

### 场景设置

我们有一个水库，其目标是维持水位在 **10米**。水库的入流是随时间变化的（模拟一个洪水过程）。我们需要设计一个控制器来操作闸门，以稳定水位。

In [ ]:
# 定义一个模拟的洪水过程作为入流预测
# 这代表了我们的ARIMAForecaster能够提供的完美预测
sim_duration_steps = 100
inflow_forecast = 50 + 80 * np.sin(np.linspace(0, 2 * np.pi, sim_duration_steps))
inflow_forecast[inflow_forecast < 50] = 50 # 确保基线流量不小于50

plt.figure(figsize=(10, 4))
plt.plot(inflow_forecast)
plt.title('预测的入库流量 (洪水过程)')
plt.xlabel('时间步')
plt.ylabel('流量 (m^3/s)')
plt.show()

### 运行仿真 A: PID 控制器

In [ ]:
def run_simulation(controller, inflow_forecast):
    harness = SimulationHarness(config={'dt': 3600, 'duration': 3600 * len(inflow_forecast)})
    
    # 组件
    reservoir = Reservoir("res", {'volume': 10e6, 'water_level': 10.0}, {'surface_area': 1e6})
    gate = Gate("gate", {'opening': 0.5}, {'width': 5, 'discharge_coeff': 0.8})
    control_agent = LocalControlAgent(
        agent_id="control_agent",
        controlled_object=gate,
        controller=controller,
        setpoint_key='opening',
        observation_map={'water_level': 'water_level'},
        actuator_map={'opening': 'opening'}
    )
    
    harness.add_component(reservoir)
    harness.add_component(gate)
    harness.add_agent(control_agent)
    harness.add_connection("res", "gate")
    harness.build()
    
    history = []
    for inflow in inflow_forecast:
        reservoir.set_inflow(inflow)
        # 在MPC的情况下，我们需要将预测传递给控制器
        if isinstance(controller, MPCController):
            obs_for_mpc = reservoir.get_state()
            obs_for_mpc['inflow_forecast'] = inflow_forecast # 简化：提供完整的未来预测
            control_agent.set_observation_override(obs_for_mpc)
        
        harness.step()
        history.append(reservoir.get_state())
        
    return pd.DataFrame(history)

# 1. 设置 PID 控制器
pid_controller = PIDController(kp=0.5, ki=0.1, kd=0.05, setpoint=10.0)

# 2. 运行仿真
print("正在运行 PID 控制器仿真...")
pid_results = run_simulation(pid_controller, inflow_forecast)
print("PID 仿真完成！")

### 运行仿真 B: MPC 控制器

In [ ]:
# 1. 设置 MPC 控制器
mpc_config = {
    "target_level": 10.0,
    "q_weight": 1.0,
    "r_weight": 0.2,
    "bounds": (0.0, 1.0),
    "max_outflow": 200.0
}
mpc_controller = MPCController(horizon=15, dt=3600, config=mpc_config)

# 2. 运行仿真
print("\n正在运行 MPC 控制器仿真...")
mpc_results = run_simulation(mpc_controller, inflow_forecast)
print("MPC 仿真完成！")

### 结果对比与分析

In [ ]:
plt.figure(figsize=(14, 7))

plt.plot(pid_results['water_level'], label='PID Controller', color='blue', linestyle='--')
plt.plot(mpc_results['water_level'], label='MPC Controller', color='green', linewidth=2)
plt.axhline(y=10.0, color='r', linestyle=':', label='目标水位 (Target Level)')

plt.title('PID vs MPC: 水位控制性能对比', fontsize=16)
plt.xlabel('时间步')
plt.ylabel('水库水位 (m)')
plt.legend()
plt.ylim(8, 12) # 限制y轴范围以便观察
plt.show()

pid_deviation = np.std(pid_results['water_level'])
mpc_deviation = np.std(mpc_results['water_level'])

print(f"PID 控制下水位的标准差: {pid_deviation:.3f}")
print(f"MPC 控制下水位的标准差: {mpc_deviation:.3f}")

从上图可以清晰地看到：

- **PID 控制器** 的表现有明显的滞后性。当洪水（高入流）来临时，水位已经显著上升后，它才开始大幅开闸放水，导致水位超调严重。而在洪水退去后，它又关闸太慢，导致水位降得过低。
- **MPC 控制器** 因为能够“预见”到未来的入流变化，所以在洪水到达之前就开始逐渐开大闸门，为即将到来的水量预腾库容。因此，它的水位波动范围要小得多，控制效果远比PID稳定。

这充分证明了将预测能力与先进控制算法相结合在现代水务管理中的巨大价值。